1. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát éves bontásban norbert2 azonosítójú ügyfél rendeléseinek értékéről!</span>

1. A lista megfelelően jelölve tartalmazza a végösszeget is!

In [ ]:
SELECT CASE WHEN GROUPING_ID(YEAR(r.REND_DATUM)) = 1 THEN 'Összesen'
            ELSE CAST(YEAR(r.REND_DATUM) AS nvarchar(4)) END,
        SUM(rt.EGYSEGAR*rt.MENNYISEG)
FROM Rendeles_tetel rt JOIN Rendeles r ON rt.SORSZAM = r.SORSZAM
WHERE r.[LOGIN] = 'norbert2'
GROUP BY ROLLUP(YEAR(r.REND_DATUM))

2. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát szállítási dátumonként, azon belül szállítási módonként az egyes rendelések összmennyiségéről!</span>

1. Csak azokat a termékeket vegyük figyelembe, amelyek mennyiségi egysége db!
2. A listát szűrjük úgy, hogy az csak a részösszeg sorokat és a végösszeget tartalmazza!

```
3. Hány olyan ügyfél van, aki még nem rendelt semmit?
```

1. Csoportosítsuk őket nem szerint, azon belül életkor szerint!
2. A lista tartalmazza a részösszegeket és a végösszeget is!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

4. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a megrendelt termékek legkisebb és legnagyobb egységáráról szállítási dátum, azon belül szállítási mód szerinti bontásban!</span>

1. A lista csak a 2015 májusi szállításokat tartalmazza!
2. Jelenítsük meg a részösszegeket és a végösszeget is!

5. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk csoportot a termékek listaára alapján a következők szerint:&nbsp;<br></span> <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Az "olcsó" termékek legyenek azok, amelyek listaára 3000 alatt van.<br>&nbsp;A "drága" termékek legyenek az 5000 felettiek, a többi legyen "közepes".</span>

1. Listázzuk az egyes csoportokat, és a csoportokba tartozó termékek darabszámát!
2. A lista jelenítse meg a végösszeget is!

In [ ]:
SELECT CASE WHEN LISTAAR < 3000 THEN 'olcsó'
                WHEN LISTAAR > 5000 THEN 'drága'
                ELSE 'közepes' END,
                COUNT(*) AS 'db'
FROM Termek
GROUP BY ROLLUP(CASE WHEN LISTAAR < 3000 THEN 'olcsó'
                WHEN LISTAAR > 5000 THEN 'drága'
                ELSE 'közepes' END) 

6. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát az ügyfelek adatairól név szerinti sorrendben.</span>

1. Minden sorban jelenjen meg a sorrend szerint előző, illetve következő ügyfél neve is.
2. Ha nincs előző vagy következő ügyfél, akkor a 'Nincs' jelenjen meg!

In [ ]:
SELECT  *,
        LAG(NEV, 1, 'Nincs') OVER(ORDER BY NEV) AS 'Előző',
        LEAD(NEV, 1, 'Nincs') OVER(ORDER BY NEV) AS 'Következő'
FROM Ugyfel
ORDER BY NEV

7. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk lekérdezést, amely megmutatja, hogy melyik termékkategóriába hány termék tartozik. A lista a kategória nevét és a darabszámot jelenítse meg.</span>

1. A lista ne tartalmazzon duplikált sorokat.
2. A feladatot partíciók segítségével oldjuk meg!

In [ ]:
SELECT  tk.KAT_NEV,
        t.LISTAAR,
        COUNT(t.TERMEKKOD) OVER(PARTITION BY tk.KAT_NEV
                            ORDER BY tk.KAT_NEV)
FROM Termek t JOIN Termekkategoria tk ON t.KAT_ID = tk.KAT_ID
-- GROUP BY tk.KAT_NEV

8. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről. Az egyes rendelési tételeket termékenként soroljuk be 4 osztályba a rendelés mennyisége alapján. Jelenítsük meg ezt az információt is egy új oszlopban, az oszlop neve legyen 'Mennyiségi kategória'.</span>

1. A lista csak a 100 Ft feletti egységárú rendelési tételeket vegye figyelembe!

In [ ]:
-- NTILE fv - az értékeket csoportokba sorolja, egyforma méretűekbe (kvantilis)

9. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, megnevezését, kategóriájának nevét, és listaárát.</span>

1. A listát egészítsük ki két új oszloppal, amelyek a kategória legolcsóbb, illetve legdrágább termékének árát tartalmazzák.
2. A két új oszlop létrehozásánál partíciókkal dolgozzunk!

10. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelésekről. A lista legyen rendezve ügyfelenként (LOGIN), azon belül a rendelés dátuma szerint. A listához készítsünk sorszámozást is. A sorszám a következő formában jelenjen meg: sorszám_év_login. Pl: 1_2015_adam1</span>

1. A számozás login-onként, azon belül rendelési évenként kezdődjön újra.
2. A sorszám oszlop neve legyen Azonosító

11. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek adatairól listaár szerint növekvő sorrendben! A lista jelenítse meg két új oszlopban a sorrend szerint előző, illetve következő termék listaárát is a termék saját kategóriájában és raktárában!</span>

1. Ahol nincs előző vagy következő érték, ott 0 jelenjen meg!
2. Az oszlopokat nevezzük el értelemszerűen!

12. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, nevét és listaárát listaár szerinti sorrendben!</span>

1. Vegyünk fel egy új oszlopot Mozgóátlag néven, amely minden esetben az aktuális termék, az előző, és a következő termék átlagárát tartalmazza!
2. A mozgóátlagot kerekítsük két tizedesre!

In [ ]:
-- fizikai ablak
SELECT TERMEKKOD, 
        MEGNEVEZES,
        LISTAAR,
        (LAG(LISTAAR) OVER (ORDER BY LISTAAR) +
        LISTAAR +
        LEAD(LISTAAR) OVER (ORDER BY LISTAAR))/3 AS 'Mozgóátlag',
        -- fizikai ablak: 3 soros
        AVG(LISTAAR) OVER(ORDER BY LISTAAR ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS 'Mozgóátlag'
FROM Termek
-- ORDER BY LISTAAR

13. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát, amely a rendelések sorszámát és a rendelés értékét tartalmazza. A listát egészítsük ki egy új oszloppal, amely minden rendelés esetén addigi rendelések összegét tartalmazza (az aktuálisat is beleértve)!</span>

1. A listát rendezzük sorszám szerint növekvő sorrendbe.
2. A lista ne tartalmazzon duplikált sorokat!

In [55]:
-- logikai ablak elejétől az aktuálisig
SELECT DISTINCT SORSZAM,
        SUM(MENNYISEG*EGYSEGAR) OVER(PARTITION BY SORSZAM) AS 'RENDELÉS Összeg',
        SUM(MENNYISEG*EGYSEGAR) OVER (ORDER BY SORSZAM RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS 'Göngyölített összeg'
FROM Rendeles_tetel
ORDER BY SORSZAM

Warning: Null value is eliminated by an aggregate or other SET operation.

(28065 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:02.218

SORSZAM,RENDELÉS Összeg,Göngyölített összeg
1,11050,11050
2,105906,116956
3,41510,158466
4,61990,220456
5,75060,295516
6,157070,452586
7,267994,720580
8,42660,763240
9,27700,790940
10,54950,845890


14. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek kódjáról, nevéről, kategória azonosítójáról, raktár azonosítójáról és listaáráról, valamint a termék adott szempontok szerinti rangsorokban elfoglalt helyezéseiről. (Szempontonként külön oszlopban, a helyezéseknél növekvő sorrendet feltételezve). A szempontok a következők legyenek: listaár, kategória szerinti listaár, és raktárkód szerinti listaár.</span>

1. Az oszlopokat nevezzük el értelemszerűen.
2. A helyezések egyenlőség esetén "sűrűn" kövessék egymást.
3. A lista legyen rendezett kategória azonosító, azon belül listaár szerint!

In [57]:
-- sorszámozás: ROW_NUMBER(), RANK(), DENSE_RANK()
SELECT  DENSE_RANK() OVER(ORDER BY LISTAAR) AS 'Listaar szerinti ssz.',
        DENSE_RANK() OVER(ORDER BY KAT_ID, LISTAAR) AS 'Kat-Listaar szerinti ssz.',
        DENSE_RANK() OVER(ORDER BY RAKTAR_KOD) AS 'Raktárkód szerinti ssz.',
        TERMEKKOD,
        MEGNEVEZES,
        KAT_ID,
        RAKTAR_KOD,
        LISTAAR
FROM Termek

(551 rows affected)

Total execution time: 00:00:00.016

Listaar szerinti ssz.,Kat-Listaar szerinti ssz.,Raktárkód szerinti ssz.,TERMEKKOD,MEGNEVEZES,KAT_ID,RAKTAR_KOD,LISTAAR
149,112,1,L023-FE301,lány alsó,35,1,390
206,119,1,F021-IN215,férfi ing,37,1,6490
216,120,1,F001-KB101,férfi szövet kabát,38,1,22000
213,122,1,F013-NA207,férfi szövet nadrág,39,1,11900
207,195,1,G064-CI401,gyermek szandál (fiú),54,1,6900
210,197,1,L082-CI419,gyermek csizma (lány),54,1,8000
195,199,1,L019-NA207,gyermek pamut nadrág (lány),56,1,1290
212,198,1,L001-KB101,gyermek kabát (lány),55,1,8900
212,198,1,G001-KB101,gyermek kabát (fiú),55,1,8900
217,364,1,N001-KB101,női szövet kabát,107,1,24000


15. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről, amely minden sor esetén göngyölítve tartalmazza az ügyfél adott rendelési tételig meglévő rendelési tételeinek összértékét!</span>

1. Az új oszlop neve legyen Eddigi rendelési tételek összértéke!
2. Az ügyfél neve is jelenjen meg!